In [ ]:
import json

def id_2_coords(id):
    if id == 0:
        return depot['x_coord'], depot['y_coord']
    else:
        return patients[str(id)]['x_coord'], patients[str(id)]['y_coord']

instance = 9

filename = f'train_{instance}.json'

with open('../data/' + filename, 'r') as file:
    data = json.load(file)

depot = data['depot']
patients = data['patients']

with open('../solutions/' + filename, 'r') as file:
    data = json.load(file)

chromosomes = []
for chromosome in data:

    solution = chromosome['phenotype']
    fitness = chromosome['fitness']
    time_unfitness = chromosome['time_unfitness']
    strain_unfitness = chromosome['strain_unfitness']

    # Create arcs
    visits = []
    for route in solution:
        if len(route) == 0:
            continue
        visits.append(0)
        visits.extend(route)
    visits.append(0)

    arcs = list(zip(visits[:-1], visits[1:]))

    arcs = [(id_2_coords(a), id_2_coords(b)) for a, b in arcs]

    chromosomes.append(dict(
        arcs=arcs,
        genotype=chromosome['genotype'],
        fitness=fitness,
        time_unfitness=time_unfitness,
        strain_unfitness=strain_unfitness
    ))

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from itertools import cycle

solution_nr = 0
chromosome = sorted(chromosomes, key=lambda x: (x['time_unfitness'], x['strain_unfitness'], x['fitness']))[solution_nr]

depot_x, depot_y = depot['x_coord'], depot['y_coord']
x_coords, y_coords = zip(*[(patient['x_coord'], patient['y_coord']) for patient in patients.values()])

patient_hover_texts = [f"Patient {pid}:<br>Location: ({patient['x_coord']}, {patient['y_coord']})<br>Time Window: [{patient['start_time']}, {patient['end_time']}]<br>Visited by nurse {chromosome['genotype'][int(pid)-1]}" 
                       for pid, patient in patients.items()]

# Create a scatter plot for patients
patients_scatter = go.Scatter(x=x_coords, y=y_coords, mode='markers', marker=dict(symbol='circle', color='darkgrey'), name='Patients', text=patient_hover_texts, hoverinfo='text')


# Create a scatter plot for the depot
depot_scatter = go.Scatter(x=[depot_x], y=[depot_y], mode='markers', marker=dict(symbol='square', color='black'), name='Depot')


# Create the figure and add the scatter plots
fig = go.Figure()
fig.add_trace(patients_scatter)
fig.add_trace(depot_scatter)
from itertools import cycle

# Generate a list of distinct colors
num_routes = 25  # Or however many routes you have


colors = px.colors.qualitative.Light24
colors.append('#1CA71C')
colors.append('#FFA500')
colors = iter(cycle(colors))
# Current color

color_ind = 1



color = next(colors)
for start, end in chromosome['arcs']:
    if (start[0] == depot_x and start[1] == depot_y):  # Start at the depot
        color = next(colors)  # Change color for the new route
    fig.add_trace(go.Scatter(x=[start[0], end[0]], y=[start[1], end[1]], mode='lines', line=dict(color=color), legendgroup=f'R{color_ind}'))

# Set axis labels
fig.update_layout(
    xaxis_title='X Coordinate', 
    yaxis_title='Y Coordinate', 
    title=f"Top {solution_nr+1} Solution of Instance {instance} with Fitness={chromosome['fitness']:.2f}, Time Unfitness={chromosome['time_unfitness']:.2f}, Strain Unfitness={chromosome['strain_unfitness']:.2f}", 
    plot_bgcolor='white', 
    paper_bgcolor='white',
    height=600,
)

# Show the plot
fig.show()

In [ ]:
for arc in arcs:
    if (2, 40) in arc:
        print(arc)

In [ ]:
colors = px.colors.qualitative.Light24


In [ ]:
colors[0]

In [ ]:
arcs

In [ ]:
data['genotype']